<a href="https://colab.research.google.com/github/minjeon99/X10/blob/main/sleepy_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import sys
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch.nn as nn
import torch.optim as optim
import torchaudio
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

  Preparing metadata (setup.py) ... done
  Created wheel for textgrid: filename=TextGrid-1.6.1-py3-none-any.whl size=10147 sha256=0310f5dcac628fd319e9f344af2e84df35f5f9ee9d75d05d111bd3efbe051e60
  Stored in directory: /root/.cache/pip/wheels/23/41/f2/e2ef1817bd163de3c21dd078966bdd71bd5c4455841f4ec016
Successfully built textgrid


In [ ]:
data = pd.read_csv('/content/emov_db.csv')

In [ ]:
#!pip install textgrid
from emov_mfa_alignment import Emov
dataset = Emov()
dataset.download()
dataset.prepare_mfa()

download successed:bea_Amused.tar.gz
download successed:bea_Angry.tar.gz
download successed:bea_Disgusted.tar.gz
download successed:bea_Neutral.tar.gz
download successed:bea_Sleepy.tar.gz
download successed:jenie_Amused.tar.gz
download successed:jenie_Angry.tar.gz
download successed:jenie_Disgusted.tar.gz
download successed:jenie_Neutral.tar.gz
download successed:jenie_Sleepy.tar.gz
download successed:josh_Amused.tar.gz
download successed:josh_Neutral.tar.gz
download successed:josh_Sleepy.tar.gz
download successed:sam_Amused.tar.gz
download successed:sam_Angry.tar.gz
download successed:sam_Disgusted.tar.gz
download successed:sam_Neutral.tar.gz
download successed:sam_Sleepy.tar.gz
download successed:cmuarctic.data


# 데이터 로드

In [ ]:
data.head()

,file_path,label
0,/content/EMOV-DB/1/anger_29-56_0042.wav,anger
1,/content/EMOV-DB/1/sleepiness_225-252_0246.wav,sleepiness
2,/content/EMOV-DB/1/sleepiness_449-476_0456.wav,sleepiness
3,/content/EMOV-DB/1/amused_16-28_0021.wav,amused
4,/content/EMOV-DB/1/neutral_141-168_0164.wav,neutral


In [ ]:
dataset_path = "/content/EMOV-DB"

# 파일, 라벨 추출
data = []
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.wav'):
                label = file.split('_')[0]  # Sleepy_0001.wav → Sleepy
                data.append({'file_path': os.path.join(folder_path, file), 'label': label})

In [ ]:
for i in data:
  i['label'] = i['label'].lower()

data_df.label.value_counts()

,count
label,
sleepiness,1721
neutral,1568
amused,1317
anger,1268
disgust,1019


In [ ]:
#data_df = pd.DataFrame(data)
#data_df.to_csv('emov_db.csv', index=False)

# Model

## Librosa + RF

### 전처리

- 특징 추출

In [ ]:
# MFCC 특징 추출 함수
def extract_features_mfcc(file_path, n_mfcc=13):
    audio, sr = librosa.load(file_path, sr=None)  # 원본 샘플링 속도 유지
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc, axis=1)  # 평균값으로 축소

In [ ]:
# 모든 파일에 대해 특징 추출
features = []
labels = []

for item in np.array(data):
    features.append(extract_features_mfcc(item[0]))
    labels.append(item[1])

# numpy 배열로 변환
X = np.array(features)
y = np.array(labels)

- 레이블 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [ ]:
le.classes_

array(['amused', 'anger', 'disgust', 'neutral', 'sleepiness'],
      dtype='<U10')

### binary classification

In [ ]:
from sklearn.model_selection import train_test_split

# sleepiness(4) → True, else(0,1,2,3) → False
y_encoded_bi = (y_encoded == 4)

X_train, X_test, y_train_bi, y_test_bi = train_test_split(X, y_encoded_bi, test_size=0.2, stratify=y_encoded_bi, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf_bi = RandomForestClassifier(random_state=42)
rf_bi.fit(X_train, y_train_bi)

y_pred_bi = rf_bi.predict(X_test)
print("Accuracy:", accuracy_score(y_test_bi, y_pred_bi))
print("RandomForest Classification Report:\n", classification_report(y_test_bi, y_pred_bi))

Accuracy: 0.939811457577955
RandomForest Classification Report:
               precision    recall  f1-score   support

       False       0.95      0.97      0.96      1035
        True       0.91      0.84      0.87       344

    accuracy                           0.94      1379
   macro avg       0.93      0.91      0.92      1379
weighted avg       0.94      0.94      0.94      1379



### Sample

1. sleepiness sample test

In [ ]:
from IPython.display import Audio
sample_idx = data[data['label']=='sleepiness'].sample(1).index
sample_data = data.iloc[sample_idx]
sample_path = sample_data.file_path.values[0]
sample_audio, sample_sr = librosa.load(sample_path, sr=None)
Audio(sample_audio, rate=sample_sr)

In [ ]:
sample_X = np.array(extract_features_mfcc(sample_path)).reshape(1,-1)
sample_proba = rf_bi.predict_proba(sample_X)[0]
sample_pred = rf_bi.predict(sample_X)
classes = {0:'False', 1:'True'}
print(f"{sample_proba[sample_proba.argmax()]}의 확률로 {sample_pred.item()}")

0.97의 확률로 True


2. other sample test

In [ ]:
sample_idx = data[data['label']!='sleepiness'].sample(1).index
sample_data = data.iloc[sample_idx]
sample_path = sample_data.file_path.values[0]
sample_audio, sample_sr = librosa.load(sample_path, sr=None)
Audio(sample_audio, rate=sample_sr)

In [ ]:
sample_X = np.array(extract_features_mfcc(sample_path)).reshape(1,-1)
sample_proba = rf_bi.predict_proba(sample_X)[0]
sample_pred = rf_bi.predict(sample_X)
classes = {0:'False', 1:'True'}
print(f"{sample_proba[sample_proba.argmax()]}의 확률로 {sample_pred.item()}")

0.84의 확률로 False
